![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/46.LLMLoader.ipynb)

# LLMLoader



LLMLoader is designed to interact with a LLMs that are converted into gguf format. This module allows using John Snow Labs' licensed LLMs at various sizes that are finetuned on medical context for certain tasks. It provides various methods for setting parameters, loading models, generating text, and retrieving metadata. The LLMLoader includes methods for setting various parameters such as input prefix, suffix, cache prompt, number of tokens to predict, sampling techniques, temperature, penalties, and more. Overall, the LLMLoader provides a flexible and extensible framework for interacting with language models in a Python and Scala environment using PySpark and Java.

## Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.4.0
Spark NLP_JSL Version : 5.4.0


# Pretrained Models



| Model Name              | Description |
|-------------------------|-------------|
|[JSL_MedS_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q16_v1_en.html)      | This LLM model is trained to perform Summarization and Q&A based on a given context |
|[JSL_MedS_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q8_v1_en.html)       | This LLM model is trained to perform Summarization and Q&A based on a given context |
|[JSL_MedS_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q4_v1_en.html)       | This LLM model is trained to perform Summarization and Q&A based on a given context |
|[JSL_MedM_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q16_v1_en.html)      | This LLM model is trained to perform Q&A, Summarization, RAG, and Chat |
|[JSL_MedM_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q8_v1_en.html)       | This LLM model is trained to perform Q&A, Summarization, RAG, and Chat |
|[JSL_MedM_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q4_v1_en.html)       | This LLM model is trained to perform Q&A, Summarization, RAG, and Chat |
|[JSL_MedSNer_ZS_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zh_q16_en.html)| This LLM model is trained to extract and link entities in a document |
|[JSL_MedSNer_ZS_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zh_q8_en.html) | This LLM model is trained to extract and link entities in a document |
|[JSL_MedSNer_ZS_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zh_q4_en.html) | This LLM model is trained to extract and link entities in a document |


We recommend using 8b quantised versions of the models as the qualitative performance difference between q16 and q8 versions is very negligible.

## JSL_MedS


This LLM model is trained to perform Summarization and Q&A based on a given context.

In [6]:
from sparknlp_jsl.llm import LLMLoader

llm_loader_pretrained = LLMLoader(spark).pretrained("jsl_meds_q8_v1", "en", "clinical/models")

In [ ]:
ptompt = """
Based on the following text, what age group is most susceptible to breast cancer?

## Text:
The exact cause of breast cancer is unknown. However, several risk factors can increase your likelihood of developing breast cancer, such as:
- A personal or family history of breast cancer
- A genetic mutation, such as BRCA1 or BRCA2
- Exposure to radiation
- Age (most commonly occurring in women over 50)
- Early onset of menstruation or late menopause
- Obesity
- Hormonal factors, such as taking hormone replacement therapy
"""

response = llm_loader_pretrained.generate(ptompt)

In [9]:
response

' The age group most susceptible to breast cancer, as mentioned in the text, is women over the age of 50.'

## JSL_MedM


This LLM model is trained to perform Q&A, Summarization, RAG, and Chat.

In [8]:
from sparknlp_jsl.llm import LLMLoader

llm_loader_pretrained = LLMLoader(spark).pretrained("jsl_medm_q8_v1", "en", "clinical/models")

In [ ]:
ptompt = """
A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.
Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

response = llm_loader_pretrained.generate(ptompt)

In [11]:
print(response)

The correct answer is E: Nitrofurantoin.

The patient is 22 weeks pregnant and has symptoms of burning upon urination, which is a common symptom of urinary tract infection (UTI). Nitrofurantoin is a first-line antibiotic for uncomplicated UTI in pregnant women.


In [ ]:
### Output:
"""The correct answer is E: Nitrofurantoin.

The patient is presenting with symptoms of urinary tract infection (UTI), which is common during pregnancy. Nitrofurantoin is a first-line antibiotic for uncomplicated UTI during pregnancy. It is safe and effective in treating UTI during pregnancy and has been used for many years without any adverse effects on the fetus.
"""

## JSL_MedSNer


This LLM model is trained to extract and link entities in a document.
Users needs to define an input schema as explained in the example section.
Drug is defined as a list which tells the model that there could be multiple drugs in the document and it has to extract all of them.
Each drug has properties like "name" and "reaction". Since "name" is only one, it is a string, but there could be multiple reactions, hence it is a list.
Similarly, users can define any schema for any type of entity.

In [12]:
from sparknlp_jsl.llm import LLMLoader

llm_loader_pretrained = LLMLoader(spark).pretrained("jsl_medner_zs_q16", "en", "clinical/models")

In [13]:
ptompt = """
### Template:
{
    "drugs": [
        {
            "name": "",
            "reactions": []
        }
    ]
}
### Text:
I feel a bit drowsy & have a little blurred vision , and some gastric problems .
I 've been on Arthrotec 50 for over 10 years on and off , only taking it when I needed it .
Due to my arthritis getting progressively worse , to the point where I am in tears with the agony.
Gp 's started me on 75 twice a day and I have to take it every day for the next month to see how I get on , here goes .
So far its been very good , pains almost gone , but I feel a bit weird , did n't have that when on 50.
"""

response = llm_loader_pretrained.generate(ptompt)

In [ ]:
response

In [ ]:
print(response)

In [ ]:
####### Model: JSL_MedSNer_ZS_q16_v1
### Output:
"""
{
    "drugs": [
        {
            "name": "Arthrotec",
            "reactions": [
                "drowsy",
                "blurred vision",
                "gastric problems"
            ]
        }
    ]
}
"""